# Mixing Languages

Durante el indexado se realizan una serie de operaciones de análisis textual de los campos de los documentos. 
Este análisis se realiza mediante los analizadores que especifiquemos a la hora de crear el índice. 

Para que estos analizadores funcionen y hagan su labor correctamente, es necesario tener en cuenta el idioma del texto a analizar.

Elasticsearch incorpora de fábrica una serie de analizadores para los siguientes idiomas:

Arabic, Armenian, Basque, Brazilian, Bulgarian, Catalan, Chinese, Czech, Danish, Dutch, English, Finnish, French, Galician, German, Greek, Hindi, Hungarian, Indonesian, Irish, Italian, Japanese, Korean, Kurdish, Norwegian, Persian, Portuguese, Romanian, Russian, Spanish, Swedish, Turkish, y Thai.

Estos analizadores realizan las siguientes funciones:
* Toquenizado: extraen las palabras del texto:  The quick brown foxes → [The, quick, brown, foxes]
* Pasar a minúsculas las palabras toquenizadas: The → the
* Eliminar las stop words: [The, quick, brown, foxes] → [quick, brown, foxes]
* Stemming: Extraer las raíces léxicas de las palabras toquenizadas foxes → fox
* Otras operaciones específicas de cada idioma. Por ejemplo:
* El analizador para Inglés elimina el posesivo 's: John's → john
* El analizador para francés elimina las eliesons como l', qu' o tildes como ^, ":  l'église → eglis

Si no se especifica analizador, Elasticsearch aplica un analizador estandar que no tiene en cuenta el idioma del documento.


## Errores más comunes al mezclar idiomas:

### Indexado: 
A la hora de insertar documentos multilenguaje en Elasticsearch hay tres alternativas:
* Un solo idioma por documento que pueden contener fragmentos en otros idiomas.
* Un idioma predominante por campo que puede contener fragmentos de otros idiomas.
* Mezcla de idiomas por campo.

En todas las alternativas el objetivo que se pretende conseguir (no siempre es posible) es mantener los idiomas separados. Mezclar lenguajes en el mismo índice inverso puede ser problemático.

#### Stemming incorrecto: 
Cada idioma tiene sus reglas diferentes para hacer el stemming. Aplicar el mismo stemming para distintos idiomas, algunas palabras se analizaran correctamente y otras no. Extraer la misma raíz para palabras de distintos idiomas que tienen diferente significado o al revés, …

Aplicar diferentes stemmers sobre las mismas palabras puede incurrir igualmente en errores. Aplicar un stemmer al resultado de las raíces del anterior da resultados erróneos. Aplicar siempre la regla only-one-stemmer.

#### Frecuencias incorrectas en el índice inverso:
Cuanto más alta es la frecuencia de aparición de un término en una colección de documentos, menos peso tendrá el término. 

Para tener una mayor precisión en el cálculo de relevancia, es necesario tener una buena estadística de frecuencias de términos. 

Los stemmers tienen mucha importancia en la generación de estas frecuencias. Aplicar un stemmer en inglés sobre documentos que tienen fragmentos de textos en alemán, dará más relevancia a los términos en alemán frente a los predominantes en inglés.

### Durante las consultas: 
A la hora de consultar, es fundamental identificar cual es el idioma principal del usuario:
* Usuarios que buscan palabras en su idioma principal.
* Usuarios que buscan palabras en otro idioma pero esperan resultados en su lenguaje principal.
* Usuarios que buscan palabras en otro idioma y esperan resultados en ese idioma (personas bilingües o que visitan una página que está en un idioma que no es el suyo)

Dependiendo de lo anterior se pueden devolver resultados sólo del lenguaje de preferencia del usuario, o juntar esos resultados con los de otros lenguajes dando preferencia al idioma del usuario.


### Alternativas:

#### Un sólo idioma por documento
* Índices separados por idiomas. Los documentos de cada idioma se indexan en el índice de su idioma. Esta solución elimina los problemas con las frecuencias de términos y de stemming.
* Para hacer una consulta sobre varios idiomas, se pueden realizar la consultas sobre varios índices a la vez. Para darle más relevancia a un idioma en la bísqueda podemos utilizar el parámetro indices_boost.


#### Un idioma por campo

* Cada idioma tiene su propio campo dentro del documento que es analizado acorde al lenguaje que contiene. De esta forma se mantienen limpias la frecuencias de los términos para cada idioma. 
* Estos índices pueden consultarse de forma separada o si es necesario hacer búsquedas en varios idiomas se pueden consultar tantos campos como en tantos idiomas sean necesarios.
* Además se puede especificar diferente boost para cada campo.


#### Campos con idiomas mezclados

Tenemos texto con varios idiomas mezclados. En estos casos es complicado manejar correctamente el análisis. 

Algunas alternativas son:
* Separar el texto en diferentes campos:
    * Para ello utilizaremos detectores de idioma para separar las frases de cada idioma del texto original. 
    * Una vez separadas indexamos cada frase en su campo.
* Analizar el textos varias veces:
    * Creamos un campo con el valor original y el analizador estandar por defecto. 
    * Utilizamos un multi-field para cada idioma con el analizador adecuado.
